
# **COMP9414 Artificial Intelligence**
## Assignment 1: Constraint Satisfaction Search

@Authors: **Wayne Wobcke, Alfred Krzywicki, Stefano Mezza**

**Due Date:** Week 5, Friday, October 17, 5.00pm

### Objective

This assignment concerns developing optimal solutions to a scheduling problem inspired by the scenario of a manufacturing plant that has to fulfil multiple customer orders with varying deadlines, but where there may be constraints on tasks and on relationships between tasks. Any number of tasks can be scheduled at the same time, but it is possible that some tasks cannot be finished before their deadline. A task finishing late is acceptable, however incurs a cost, which for this assignment is a simple (dollar) amount per hour that the task is late.

A *fuzzy scheduling* problem in this scenario is simplified by ignoring customer orders and having just one machine and a number of *tasks*, each with a fixed duration in hours. Each task must start and finish on the same day, within working hours (9am to 5pm). In addition, there can be *constraints*, both on single tasks and between two tasks. One type of constraint is that a task can have a deadline, which can be “hard” (the deadline must be met in any valid schedule) or “soft” (the task may be finished late &ndash; though still at or before 5pm &ndash; but with a “cost” per hour for missing the deadline). The aim is to develop an overall schedule for all the tasks (in a single week) that minimizes the total cost of all the tasks that finish late, provided that all the hard constraints on tasks are satisfied.

More technically, this assignment is an example of a *constraint optimization problem* (or *constrained optimization problem*), a problem that has constraints like a standard Constraint Satisfaction Problem (CSP), but also a *cost* associated with each solution. For this assignment, we will use a *greedy* algorithm to find optimal solutions to fuzzy scheduling problems that are specified as text strings. However, unlike the greedy search algorithm described in the lectures on search, this greedy algorithm has the property that it is guaranteed to find an optimal solution for any problem (if a solution exists).

The assignment will use the AIPython code of Poole & Mackworth. You are given code to translate fuzzy scheduling problems specified as text strings into CSPs with a cost, and you are given code for several constraint solving algorithms &ndash; based on domain splitting and arc consistency, and based on depth-first search. The assignment will be to implement some missing procedures and to analyse the performance of the constraint solving methods, both analytically and experimentally.

### Submission Instructions

- This is an individual assignment.

- Write your answers in **this** notebook and submit **this** notebook on Moodle under **Assignment 1**.

- Name your submission ``<zid>-<firstname>-<lastname>.ipynb`` where ``<firstname>-<lastname>`` is your **real** (not Moodle) name.

- Make sure you set up AIPython (as done below) so the code can be run on either CSE machines or a marker's own machine.

- Do not submit any AIPython code. Hence do not change any AIPython code to make your code run.

- Make sure your notebook runs cleanly (restart the kernel, clear all outputs and run each cell to check).

- After checking that your notebook runs cleanly, run all cells and submit the notebook **with** the outputs included (do not submit the empty version).

- Make sure images (for plots/graphs) are **included** in the notebook you submit (sometimes images are saved on your machine but are not in the notebook).

- Do not modify the existing code in this notebook except to answer the questions. Marks will be given as and where indicated.

- If you want to submit additional code (e.g. for generating plots), add that at the end of the notebook.

- **Important: Do not distribute any of this code on the Internet. This includes ChatGPT. Do not put this assignment into any LLM.**

### Late Penalties

Standard UNSW late penalties apply (5% of the value of the assignment per day or part day late).

**Note:** Unlike the CSE systems, there is no grace period on Moodle. The due date and time is 5pm **precisely** on Friday October 17.

**Important: You can submit as many times as you want before the due date, but if you do submit before the due date, you cannot submit on Moodle after the due date. If you do not submit before the due date, you can submit on Moodle after the due date.**

### Plagiarism

Remember that ALL work submitted for this assignment must be your own work and no sharing or copying of code or answers is allowed. You may discuss the assignment with other students but must not collaborate on developing answers to the questions. You may use code from the Internet only with suitable attribution of the source. You may not use ChatGPT or any similar software to generate any part of your explanations, evaluations or code. Do not use public code repositories on sites such as github or file sharing sites such as Google Drive to save any part of your work &ndash; make sure your code repository or cloud storage is private and do not share any links. This also applies after you have finished the course, as we do not want next year’s students accessing your solution, and plagiarism penalties can still apply after the course has finished.

All submitted assignments will be run through plagiarism detection software to detect similarities to other submissions, including from past years. You should **carefully** read the UNSW policy on academic integrity and plagiarism (linked from the course web page), noting, in particular, that collusion (working together on an assignment, or sharing parts of assignment solutions) is a form of plagiarism.

Finally, do not use any contract cheating “academies” or online “tutoring” services. This counts as serious misconduct with heavy penalties up to automatic failure of the course with 0 marks, and expulsion from the university for repeat offenders.

### Fuzzy Scheduling

A CSP for this assignment is a set of variables representing tasks, binary constraints on pairs of tasks, and unary constraints (hard or soft) on tasks. The domains are all the working hours in one week, and a task duration is in hours. Days are represented (in the input and output) as strings ‘mon’, ‘tue’, ‘wed’, ‘thu’ and ‘fri’, and times are represented as strings ‘9am’, ‘10am’, ‘11am’, ‘12pm’, ‘1pm’, ‘2pm’, ‘3pm’, ‘4pm’ and ‘5pm’. The only possible values for the start and end times of a task are combinations of a day and times, e.g. ‘mon 9am’. Each task name is a string (with no spaces), and the only soft constraints are the soft deadline constraints.

There are three types of constraint:

- **Binary Constraints:** These specify a hard requirement for the relationship between two tasks.
- **Hard Domain Constraints:** These specify hard requirements for the tasks themselves.
- **Soft Deadline Constraints:** These constraints specify that a task may finish late, but with a given cost.

Each soft constraint has a function defining the *cost* associated with violating the preference, that the constraint solver must minimize, while respecting all the hard constraints. The *cost* of a solution is simply the sum of the costs for the soft constraints that the solution violates (and is always a non-negative integer).

This is the list of possible constraints for a fuzzy scheduling problem (comments below are for explanation and do **not** appear in the input specification; however, the code we supply *should* work with comments that take up a full line): 

```
# binary constraints
constraint, ⟨t1⟩ before ⟨t2⟩           # t1 ends when or before t2 starts
constraint, ⟨t1⟩ after ⟨t2⟩            # t1 starts after or when t2 ends
constraint, ⟨t1⟩ same-day ⟨t2⟩         # t1 and t2 are scheduled on the same day
constraint, ⟨t1⟩ starts-at ⟨t2⟩        # t1 starts exactly when t2 ends

# hard domain constraints
domain, ⟨t⟩, ⟨day⟩, hard                                    # t starts on given day at any time
domain, ⟨t⟩, ⟨time⟩, hard                                   # t starts at given time on any day
domain, ⟨t⟩, starts-before ⟨day⟩ ⟨time⟩, hard               # t starts at or before day, time
domain, ⟨t⟩, starts-after ⟨day⟩ ⟨time⟩, hard                # t starts at or after day, time
domain, ⟨t⟩, ends-before ⟨day⟩ ⟨time⟩, hard                 # t ends at or before day, time
domain, ⟨t⟩, ends-after ⟨day⟩ ⟨time⟩, hard                  # t starts at or after day, time
domain, ⟨t⟩, starts-in ⟨day1⟩ ⟨time1⟩-⟨day2⟩ ⟨time2⟩, hard  # day-time range for start time; includes day1, time1 and day2, time2
domain, ⟨t⟩, ends-in ⟨day1⟩ ⟨time1⟩-⟨day2⟩ ⟨time2⟩, hard    # day-time range for end time; includes day1, time1 and day2, time2
domain, ⟨t⟩, starts-before ⟨time⟩, hard                     # t starts at or before time on any day
domain, ⟨t⟩, ends-before ⟨time⟩, hard                       # t ends at or before time on any day
domain, ⟨t⟩, starts-after ⟨time⟩, hard                      # t starts at or after time on any day
domain, ⟨t⟩, ends-after ⟨time⟩, hard                        # t ends at or after time on any day

# soft deadline constraint
domain, ⟨t⟩, ends-by ⟨day⟩ ⟨time⟩ ⟨cost⟩, soft          # cost per hour of missing deadline
```

The input specification will consist of several “blocks”, listing the tasks, binary constraints, hard unary constraints and soft deadline constraints for the given problem. A “declaration” of each task will be included before it is used in a constraint. A sample input specification is as follows. Comments are for explanation and do **not** have to be included in the input.

```
# two tasks with two binary constraints and soft deadlines
task, t1 3
task, t2 4
# two binary constraints
constraint, t1 before t2
constraint, t1 same-day t2
# domain constraint
domain, t2 mon
# soft deadline constraints
domain, t1 ends-by mon 3pm 10
domain, t2 ends-by mon 3pm 10
```

## Preparation

### 1. Set up AIPython

You will need AIPython for this assignment. To find the aipython files, the aipython directory has to be added to the Python path.

Do this temporarily, as done here, so we can find AIPython and run your code (you will not submit any AIPython code).

You can add either the full path (using `os.path.abspath`), or as in the code below, the relative path.

In [1]:
import sys, os

aipython_path = os.path.join(os.getcwd(), 'aipython', 'aipython')

if aipython_path not in sys.path:
    sys.path.append(aipython_path)
    
sys.path # check that aipython is now on the path

['C:\\Users\\Lenovo\\UNSW\\comp9414',
 'd:\\py\\Anaconda3\\python39.zip',
 'd:\\py\\Anaconda3\\DLLs',
 'd:\\py\\Anaconda3\\lib',
 'd:\\py\\Anaconda3',
 '',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'd:\\py\\Anaconda3\\lib\\site-packages',
 'd:\\py\\Anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg',
 'd:\\py\\Anaconda3\\lib\\site-packages\\win32',
 'd:\\py\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'd:\\py\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'd:\\py\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Lenovo\\.ipython',
 'C:\\Users\\Lenovo\\UNSW\\comp9414\\aipython\\aipython']

### 2. Representation of Day Times

Input and output are day time strings such as 'mon 10am' or a range of day time strings such as 'mon 10am-mon 4pm'.

The CSP will represent these as integer hour numbers in the week, ranging from 0 to 39.

The following code handles the conversion between day time strings and hour numbers.

In [2]:
# -*- coding: utf-8 -*-
    
""" day_time string format is a day plus time, e.g. Mon 10am, Tue 4pm, or just Tue or 4pm
    if only day or time, returns day number or hour number only
    day_time strings are converted to and from integer hours in the week from 0 to 39
"""
class Day_Time():
    num_hours_in_day = 8
    num_days_in_week = 5
    
    def __init__(self):
        self.day_names = ['mon','tue','wed','thu','fri']
        self.time_names = ['9am','10am','11am','12pm','1pm','2pm','3pm','4pm']
    
    def string_to_week_hour_number(self, day_time_str):
        """ convert a single day_time into an integer hour in the week """
        value = None
        value_type = None
        day_time_list = day_time_str.split()
        if len(day_time_list) == 1:
            str1 = day_time_list[0].strip()
            if str1 in self.time_names: # this is a time
                value = self.time_names.index(str1)
                value_type = 'hour_number'
            else:
                value = self.day_names.index(str1) # this is a day
                value_type = 'day_number'
            # if not day or time, throw an exception
        else:
            value = self.day_names.index(day_time_list[0].strip())*self.num_hours_in_day \
                + self.time_names.index(day_time_list[1].strip())
            value_type = 'week_hour_number'
        return (value_type, value)

    def string_to_number_set(self, day_time_list_str):
        """ convert a list of day-times or ranges 'Mon 9am, Tue 9am-Tue 4pm' into a list of integer hours in the week
            e.g. 'mon 9am-1pm, mon 4pm' -> [0,1,2,3,4,7]
        """
        number_set = set()
        type1 = None
        for str1 in day_time_list_str.lower().split(','):
            if str1.find('-') > 0:
                # day time range
                type1, v1 = self.string_to_week_hour_number(str1.split('-')[0].strip())
                type2, v2 = self.string_to_week_hour_number(str1.split('-')[1].strip())
                if type1 != type2: return None # error, types in range spec are different
                number_set.update({n for n in range(v1, v2+1)})
            else:
                # single day time
                type2, value2 = self.string_to_week_hour_number(str1)
                if type1 != None and type1 != type2: return None # error: type in list is inconsistent
                type1 = type2
                number_set.update({value2})
        return (type1, number_set)
    
    # convert integer hour in week to day time string
    def week_hour_number_to_day_time(self, week_hour_number):
        hour = self.day_hour_number(week_hour_number)
        day = self.day_number(week_hour_number)
        return self.day_names[day]+' '+self.time_names[hour]
    
    # convert integer hour in week to integer day and integer time in day
    def hour_day_split(self, week_hour_number):
        return (self.day_hour_number(week_hour_number), self.day_number(week_hour_number))
    
    # convert integer hour in week to integer day in week
    def day_number(self, week_hour_number):
        return int(week_hour_number / self.num_hours_in_day)
    
    # convert integer hour in week to integer time in day
    def day_hour_number(self, week_hour_number):
        return week_hour_number % self.num_hours_in_day
    
    def __repr__(self):
        day_hour_number = self.week_hour_number % self.num_hours_in_day
        day_number = int(self.week_hour_number / self.num_hours_in_day)
        return self.day_names[day_number]+' '+self.time_names[day_hour_number]

### 3. Constraint Satisfaction Problems with Costs over Tasks with Durations

Since AI Python does not provide the CSP class with an explicit cost, we implement our own class that extends `CSP`.

We also store the cost functions and the durations of all tasks explicitly in the CSP.

The durations of the tasks are used in the `hold` function to evaluate constraints.

In [3]:
from cspProblem import CSP, Constraint

# We need to override Constraint, because tasks have durations
class Task_Constraint(Constraint):
    """A Task_Constraint consists of
    * scope: a tuple of variables
    * spec: text description of the constraint used in debugging
    * condition: a function that can applied to a tuple of values for the variables
    * durations: durations of all tasks
    * func_key: index to the function used to evaluate the constraint
    """
    def __init__(self, scope, spec, condition, durations, func_key):
        super().__init__(scope, condition, spec)
        self.scope = scope
        self.condition = condition
        self.durations = durations
        self.func_key = func_key

    def holds(self, assignment):
        """returns the value of Constraint con evaluated in assignment.

        precondition: all variables are assigned in assignment
        
        CSP has only binary constraints
        condition is in the form week_hour_number1, week_hour_number2
        add task durations as appropriate to evaluate condition
        """
        if self.func_key == 'before':
            # t1 ends before t2 starts, so we need add duration to t1 assignment
            ass0 = assignment[self.scope[0]] + self.durations[self.scope[0]]
            ass1 = assignment[self.scope[1]]
        elif self.func_key == 'after':
            # t2 ends before t1 starts so we need add duration to t2 assignment
            ass0 = assignment[self.scope[0]]
            ass1 = assignment[self.scope[1]] + self.durations[self.scope[1]]
        elif self.func_key == 'starts-at':
            # t1 starts exactly when t2 ends, so we need add duration to t2 assignment
            ass0 = assignment[self.scope[0]]
            ass1 = assignment[self.scope[1]] + self.durations[self.scope[1]]
        else:
            return self.condition(*tuple(assignment[v] for v in self.scope))    
        # condition here comes from get_binary_constraint
        return self.condition(*tuple([ass0, ass1]))

# implement nodes as CSP problems with cost functions
class CSP_with_Cost(CSP):
    """ cost_functions maps a CSP var, here a task name, to a list of functions for the constraints that apply """
    def __init__(self, domains, durations, constraints, cost_functions, soft_day_time, soft_costs):
        self.domains = domains
        self.variables = self.domains.keys()
        super().__init__("title of csp", self.variables, constraints)
        self.durations = durations
        self.cost_functions = cost_functions
        self.soft_day_time = soft_day_time
        self.soft_costs = soft_costs
        self.cost = self.calculate_cost()

    # specific to fuzzy scheduling CSP problems
    def calculate_cost(self):
        total_cost = 0
        dt = Day_Time()

        for var, dom in self.domains.items():
            duration = self.durations.get(var, 0)
            per_hour_cost = int(self.soft_costs.get(var, 0))
            deadline_str = self.soft_day_time.get(var, None)

            deadline = None
            if deadline_str:
                try:
                    param_type, params = dt.string_to_number_set(deadline_str)
                    if param_type == 'week_hour_number' and params:
                        deadline = list(params)[0]
                except Exception:
                    pass

            if len(dom) == 1: 
                val = next(iter(dom))
                if deadline is not None:
                    end_time = val + duration
                    total_cost += max(0, end_time - deadline) * per_hour_cost
            else:  
                best_cost = float('inf')
                for val in dom:
                    if deadline is None:
                        cost = 0
                    else:
                        end_time = val + duration
                        cost = max(0, end_time - deadline) * per_hour_cost
                    best_cost = min(best_cost, cost)
                if best_cost != float('inf'):
                    total_cost += best_cost

        return int(total_cost)


    def __repr__(self):
        """ string representation of an arc"""
        return "CSP_with_Cost("+str(list(self.domains.keys()))+':'+str(self.cost)+")"

This formulates a solver for a CSP with cost as a search problem, using domain splitting with arc consistency to define the successors of a node.

In [4]:
from cspConsistency import Con_solver, select, partition_domain 
from searchProblem import Arc, Search_problem
from operator import eq, le, ge

# rewrites rather than extends Search_with_AC_from_CSP
class Search_with_AC_from_Cost_CSP(Search_problem):
    """ A search problem with domain splitting and arc consistency """
    def __init__(self, csp):
        self.cons = Con_solver(csp) # copy of the CSP with access to arc consistency algorithms
        self.domains = self.cons.make_arc_consistent(csp.domains)
        self.constraints = csp.constraints
        self.cost_functions = csp.cost_functions
        self.durations = csp.durations
        self.soft_day_time = csp.soft_day_time
        self.soft_costs = csp.soft_costs
        csp.domains = self.domains # after arc consistency
        self.csp = csp

    def is_goal(self, node):
        """ node is a goal if all domains have exactly 1 element """
        return all(len(node.domains[var]) == 1 for var in node.domains)

    def start_node(self):
        return CSP_with_Cost(self.domains, self.durations, self.constraints,
                             self.cost_functions, self.soft_day_time, self.soft_costs)

    def neighbors(self, node):
        """returns the neighboring nodes of node.
        """
        neighs = []
        var = select(x for x in node.domains if len(node.domains[x]) > 1) # chosen at random
        if var:
            dom1, dom2 = partition_domain(node.domains[var])
            self.display(2, "Splitting", var, "into", dom1, "and", dom2)
            to_do = self.cons.new_to_do(var, None)
            for dom in [dom1,dom2]:
                newdoms = node.domains | {var: dom} # overwrite domain of var with dom
                cons_doms = self.cons.make_arc_consistent(newdoms, to_do)
                if all(len(cons_doms[v]) > 0 for v in cons_doms):
                    # all domains are non-empty
                    # make new CSP_with_Cost node to continue the search
                    csp_node = CSP_with_Cost(cons_doms, self.durations, self.constraints,
                             self.cost_functions, self.soft_day_time, self.soft_costs)
                    neighs.append(Arc(node, csp_node))
                else:
                    self.display(2,"...",var,"in",dom,"has no solution")
        return neighs    
    
    def heuristic(self, n):
        return n.cost

### 4. Fuzzy Scheduling Constraint Satisfaction Problems

The following code sets up a CSP problem from a given specification.

Hard (unary) domain constraints are applied to reduce the domains of the variables before the constraint solver runs.

In [5]:
# domain specific CSP builder for week schedule
class CSP_builder():
    # list of text lines without comments and empty lines
    _, default_domain = Day_Time().string_to_number_set('mon 9am-fri 4pm') # should be 0,...,39
    
    # hard unary constraints: domain is a list of values, params is a single value
    # starts-before, ends-before (for starts-before duration should be 0)
    # vals in domain are actual task start/end date/time, so must be val <= whatever is in params
    def apply_before(self, param_type, params, duration, domain):
        domain_orig = domain.copy()
        param_val = params.pop()
        for val in domain_orig: # val is week_hour_number
            val1 = val + duration
            h, d = Day_Time().hour_day_split(val1)
            if param_type == 'hour_number' and h > param_val:
                if val in domain: domain.remove(val)
            if param_type == 'day_number' and d > param_val:
                if val in domain: domain.remove(val)
            if param_type == 'week_hour_number' and val1 > param_val:
                if val in domain: domain.remove(val)
        return domain
    
    def apply_after(self, param_type, params, duration, domain):
        domain_orig = domain.copy()
        param_val = params.pop()
        for val in domain_orig: # val is week_hour_number
            val1 = val + duration
            h, d = Day_Time().hour_day_split(val1)
            if param_type == 'hour_number' and h < param_val:
                if val in domain: domain.remove(val)
            if param_type == 'day_number' and d < param_val:
                if val in domain: domain.remove(val)
            if param_type == 'week_hour_number' and val1 < param_val:
                if val in domain: domain.remove(val)
        return domain
    
    # day time range only
    # includes starts-in, ends-in
    # duration is 0 for starts-in, task duration for ends-in
    def apply_in(self, params, duration, domain):
        domain_orig = domain.copy()
        for val in domain_orig: # val is week_hour_number
            # task must be within range
            if val in domain and val+duration not in params:
                domain.remove(val)
        return domain
    
    # task must start at day/time    
    def apply_at(self, param_type, param,domain):
        domain_orig = domain.copy()
        for val in domain_orig:
            h, d = Day_Time().hour_day_split(val)
            if param_type == 'hour_number' and param != h:
                if val in domain: domain.remove(val)
            if param_type == 'day_number' and param != d:
                if val in domain: domain.remove(val)
            if param_type == 'week_hour_number' and param != val:
                if val in domain: domain.remove(val)
        return domain

    # soft deadline constraints: return cost to break constraint
    # ends-by implementation: domain_dt is the day, hour from the domain
    # constr_dt is the soft const spec, dur is the duration of task
    # soft_cost is the unit cost of completion delay
    # so if the tasks starts on domain_dt, it ends on domain_dt+dur
    """
    <t> ends-by <day> <time>, both must be specified
    delay = day_hour(T2) – day_hour(T1) + 24*(D2 - D1), 
    where day_hour(9am) = 0, day_hour(5pm) = 7
    """
    def ends_by(self, domain_dt, constr_dt_str, dur, soft_cost):
        param_type,params = Day_Time().string_to_number_set(constr_dt_str)
        param_val = params.pop()
        dom_h, dom_d = Day_Time().hour_day_split(domain_dt+dur)
        if param_type == 'week_hour_number':
            con_h, con_d = Day_Time().hour_day_split(param_val)
            return 0 if domain_dt + dur <= param_val else soft_cost*(dom_h - con_h + 24*(dom_d - con_d))
        else:
           return None # not good, must be day and time
    
    def no_cost(self, day ,hour):
        return 0
    
    # hard binary constraint, the rest are implemented as gt, lt, eq
    def same_day(self, week_hour1, week_hour2):
        h1, d1 = Day_Time().hour_day_split(week_hour1)
        h2, d2 = Day_Time().hour_day_split(week_hour2)
        return d1 == d2
        
    # domain is a list of values
    def apply_hard_constraint(self, domain, duration, spec):
        tokens = func_key = spec.split(' ')
        if len(tokens) > 1:
           func_key = spec.split(' ')[0].strip()
           param_type, params = Day_Time().string_to_number_set(spec[len(func_key):])
           if func_key == 'starts-before':
              # duration is 0 for starts before, since we do not modify the time
              return self.apply_before(param_type, params, 0, domain)
           if func_key == 'ends-before':
               return self.apply_before(param_type, params, duration, domain)
           if func_key == 'starts-after':
              return self.apply_after(param_type,params,0,domain)
           if func_key == 'ends-after':
              return self.apply_after(param_type, params, duration, domain)
           if func_key == 'starts-in':
               return self.apply_in(params, 0, domain)
           if func_key == 'ends-in':
               return self.apply_in(params, duration, domain)
        else:
           # here we have task day or time, it has no func key so we need to parse again
           param_type,params = Day_Time().string_to_week_hour_number(spec)
           return self.apply_at(param_type, params, domain)
           
    def get_cost_function(self, spec):
        func_dict = {'ends-by':self.ends_by, 'no-cost':self.no_cost}
        return [func_dict[spec]]
    
    # spec is the text of a constraint, e.g. 't1 before t2'
    # durations are durations of all tasks
    def get_binary_constraint(self, spec, durations):
        tokens = spec.strip().split(' ')
        if len(tokens) != 3: return None # error in spec
        # task1 relation task2
        fun_dict = {'before':le, 'after':ge, 'starts-at':eq, 'same-day':self.same_day}
        return Task_Constraint((tokens[0].strip(), tokens[2].strip()), spec, fun_dict[tokens[1].strip()], durations, tokens[1].strip())
    
    def get_CSP_with_Cost(self, input_lines):
        # Note: It would be more elegant to make task a class but AIpython is not designed for this:
        # CSP_with_Cost inherits from CSP, which takes domains and constraints from all tasks instead of tasks themselves
        domains = dict()
        constraints = []
        cost_functions = dict()
        durations = dict() # durations of tasks
        soft_day_time = dict() # day time specs of soft constraints
        soft_costs = dict() # costs of soft constraints
        
        for input_line in input_lines:
            func_spec = None
            input_line_tokens = input_line.strip().split(',')
            if len(input_line_tokens) != 2: 
                return None # must have number of tokens = 2
            line_token1 = input_line_tokens[0].strip()
            line_token2 = input_line_tokens[1].strip()
            if line_token1 == 'task':
                tokens = line_token2.split(' ')
                if len(tokens) != 2: 
                    return None # must have number of tokens = 3
                key = tokens[0].strip()
                # check the duration and save it
                duration = int(tokens[1].strip())
                if duration > Day_Time().num_hours_in_day: 
                    return None
                durations[key] = duration
                # set zero cost function for this task as default, may add real cost later
                cost_functions[key] = self.get_cost_function('no-cost')
                soft_costs[key] = '0'
                soft_day_time[key] = 'fri 5pm'
                # restrict domain to times that are within allowed range
                # that is start 9-5, start+duration in 9-5
                domains[key] = {x for x in self.default_domain \
                                if Day_Time().day_number(x+duration) \
                                == Day_Time().day_number(x)}
            elif line_token1 == 'domain':
                tokens = line_token2.split(' ')
                if len(tokens) < 2: 
                    return None # must have number of tokens >= 2
                key = tokens[0].strip()
                # if soft constraint, it is handled differently from hard constraints
                if tokens[1].strip() == 'ends-by':
                    # need to retain day time and cost from the line
                    # must have task, 'end-by', day, time, cost
                    # or task, 'end-by', day, cost
                    # or task, 'end-by', time, cost
                    if len(tokens) != 5: 
                        return None 
                    # get the rest of the line after 'ends-by'
                    soft_costs[key] = int(tokens[len(tokens)-1].strip()) # last token is always cost
                    # pass the day time string to avoid passing param_type
                    day_time_str = tokens[2] + ' ' + tokens[3]
                    soft_day_time[key] = day_time_str
                    cost_functions[key] = self.get_cost_function(tokens[1].strip())
                else:
                    # the rest of domain spec, after key, are hard unary domain constraints
                    # func spec has day time, we also need duration
                    dur = durations[key]
                    func_spec = line_token2[len(key):].strip()
                    domains[key] = self.apply_hard_constraint(domains[key], dur, func_spec)       
            elif line_token1 == 'constraint': # all binary constraints
                constraints.append(self.get_binary_constraint(line_token2, durations))
            else:
                return None
            
        return CSP_with_Cost(domains, durations, constraints, cost_functions, soft_day_time, soft_costs)
    
def create_CSP_from_spec(spec: str):
    input_lines = list()    
    spec = spec.split('\n')
    # strip comments
    for input_line in spec:
        input_line = input_line.split('#')
        if len(input_line[0]) > 0:
            input_lines.append(input_line[0])
            print(input_line[0])
    # construct initial CSP problem
    csp = CSP_builder()
    csp_problem = csp.get_CSP_with_Cost(input_lines)
    return csp_problem

### 5. Greedy Search Constraint Solver using Domain Splitting and Arc Consistency

Create a GreedySearcher to search over the CSP.

The *cost* function for CSP nodes is used as the heuristic, but is actually a direct estimate of the total path cost function *f* used in A* Search.

In [6]:
from searchGeneric import AStarSearcher

class GreedySearcher(AStarSearcher):
    """ returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def add_to_frontier(self, path):
        """ add path to the frontier with the appropriate cost """
        # value = path.cost + self.problem.heuristic(path.end()) -- A* definition
        value = path.end().cost
        self.frontier.add(path, value)      

Run the GreedySearcher on the CSP derived from the sample input.

**Note: The solution cost will always be 0 (which is wrong for the sample input) until you write the cost function in the cell above.**

In [7]:
# Sample problem specification

sample_spec = """
# two tasks with two binary constraints and soft deadlines
task, t1 3
task, t2 4
# two binary constraints
constraint, t1 before t2
constraint, t1 same-day t2
# domain constraint
domain, t2 mon
# soft deadlines
domain, t1 ends-by mon 3pm 10
domain, t2 ends-by mon 3pm 10
"""

In [8]:
# display details (0 turns off)
Con_solver.max_display_level = 0
Search_with_AC_from_Cost_CSP.max_display_level = 2
GreedySearcher.max_display_level = 0

def test_csp_solver(searcher):
    final_path = searcher.search()
    if final_path == None:
        print('No solution')
    else:
        domains = final_path.end().domains
        result_str = ''
        for name, domain in domains.items():
            for n in domain:
                result_str += '\n'+str(name)+': '+Day_Time().week_hour_number_to_day_time(n)
        print(result_str[1:]+'\ncost: '+str(final_path.end().cost))

csp_problem = create_CSP_from_spec(sample_spec)
solver = GreedySearcher(Search_with_AC_from_Cost_CSP(csp_problem))
test_csp_solver(solver)

task, t1 3
task, t2 4
constraint, t1 before t2
constraint, t1 same-day t2
domain, t2 mon
domain, t1 ends-by mon 3pm 10
domain, t2 ends-by mon 3pm 10
t1: mon 9am
t2: mon 12pm
cost: 10


### 6. Depth-First Search Constraint Solver

The Depth-First Constraint Solver in AIPython by default uses a random ordering of the variables in the CSP.

We need to modify this code to make it compatible with the arc consistency solver.

Run the solver by calling `dfs_solve1` (first solution) or `dfs_solve_all` (all solutions).

In [9]:
num_expanded = 0
display = False

def dfs_solver(constraints, domains, context, var_order):
    """ generator for all solutions to csp
        context is an assignment of values to some of the variables
        var_order is a list of the variables in csp that are not in context
    """
    global num_expanded, display
    to_eval = {c for c in constraints if c.can_evaluate(context)}
    if all(c.holds(context) for c in to_eval):
        if var_order == []:
            print("Nodes expanded to reach solution:", num_expanded)
            yield context
        else:
            rem_cons = [c for c in constraints if c not in to_eval]
            var = var_order[0]
            for val in domains[var]:
                if display:
                    print("Setting", var, "to", val)
                num_expanded += 1
                yield from dfs_solver(rem_cons, domains, context|{var:val}, var_order[1:])

def dfs_solve_all(csp, var_order=None):
    """ depth-first CSP solver to return a list of all solutions to csp """
    global num_expanded
    num_expanded = 0
    if var_order == None:    # use an arbitrary variable order
        var_order = list(csp.domains)
    return list(dfs_solver(csp.constraints, csp.domains, {}, var_order))

def dfs_solve1(csp, var_order=None):
    """ depth-first CSP solver """
    global num_expanded
    num_expanded = 0
    if var_order == None:    # use an arbitrary variable order
        var_order = list(csp.domains)
    for sol in dfs_solver(csp.constraints, csp.domains, {}, var_order):
        return sol  # return first one

Run the Depth-First Solver on the sample problem.

**Note: Again there are no costs calculated.**

In [10]:
def test_dfs_solver(csp_problem):
    solution = dfs_solve1(csp_problem)
    if solution == None:
        print('No solution')
    else:
        result_str = ''
        for name in solution.keys():
            result_str += '\n'+str(name)+': '+Day_Time().week_hour_number_to_day_time(solution[name])
        print(result_str[1:])

# call the Depth-First Search solver
csp_problem = create_CSP_from_spec(sample_spec)
test_dfs_solver(csp_problem) # set display to True to see nodes expanded

task, t1 3
task, t2 4
constraint, t1 before t2
constraint, t1 same-day t2
domain, t2 mon
domain, t1 ends-by mon 3pm 10
domain, t2 ends-by mon 3pm 10
Nodes expanded to reach solution: 5
t1: mon 9am
t2: mon 12pm


### 7. Depth-First Search Constraint Solver using Forward Checking with MRV Heuristic

The Depth-First Constraint Solver in AIPython by default uses a random ordering of the variables in the CSP.

We redefine the `dfs_solver` methods to implement the MRV (Minimum Remaining Values) heuristic using forward checking.

Because the AIPython code is designed to manipulate domain sets, we also need to redefine `can_evaluate` to handle partial assignments.

In [11]:
num_expanded = 0
display = False

def can_evaluate(c, assignment):
    """ assignment is a variable:value dictionary
        returns True if the constraint can be evaluated given assignment
    """
    return assignment != {} and all(v in assignment.keys() and type(assignment[v]) != list for v in c.scope)

def mrv_dfs_solver(constraints, domains, context, var_order):
    """ generator for all solutions to csp.
        context is an assignment of values to some of the variables.
        var_order is a list of the variables in csp that are not in context.
    """
    global num_expanded, display
    if display:
        print("Context", context)
    to_eval = {c for c in constraints if can_evaluate(c, context)}
    if all(c.holds(context) for c in to_eval):
        if var_order == []:
            print("Nodes expanded to reach solution:", num_expanded)
            yield context
        else:
            rem_cons = [c for c in constraints if c not in to_eval] # constraints involving unset variables
            var = var_order[0]
            rem_vars = var_order[1:]
            for val in domains[var]:
                if display:
                    print("Setting", var, "to", val)
                num_expanded += 1
                rem_context = context|{var:val}
                # apply forward checking on remaining variables
                if len(var_order) > 1:
                    rem_vars_original = list((v, list(domains[v].copy())) for v in rem_vars)
                    if display:
                        print("Original domains:", rem_vars_original)
                    # constraints that can't already be evaluated in rem_cons
                    rem_cons_ff = [c for c in constraints if c in rem_cons and not can_evaluate(c, rem_context)]
                    for rem_var in rem_vars:
                        # constraints that can be evaluated by adding a value of rem_var to rem_context
                        any_value = list(domains[rem_var])[0]
                        rem_to_eval = {c for c in rem_cons_ff if can_evaluate(c, rem_context|{rem_var: any_value})}
                        # new domain for rem_var are the values for which all newly evaluable constraints hold
                        rem_vals = domains[rem_var].copy()
                        for rem_val in domains[rem_var]:
                            # no constraint with rem_var in the existing context can be violated
                            for c in rem_to_eval:
                                if not c.holds(rem_context|{rem_var: rem_val}):
                                    if rem_val in rem_vals:
                                        rem_vals.remove(rem_val)
                        domains[rem_var] = rem_vals
                        # order remaining variables by MRV
                        rem_vars.sort(key=lambda v: len(domains[v]))
                    if display:
                        print("After forward checking:", list((v, domains[v]) for v in rem_vars))
                if rem_vars == [] or all(len(domains[rem_var]) > 0 for rem_var in rem_vars):
                    yield from mrv_dfs_solver(rem_cons, domains, context|{var:val}, rem_vars)
                # restore original domains if changed through forward checking
                if len(var_order) > 1:
                    if display:
                        print("Restoring original domain", rem_vars_original)
                    for (v, domain) in rem_vars_original:
                        domains[v] = domain
            if display:
                print("Nodes expanded so far:", num_expanded)

def mrv_dfs_solve_all(csp, var_order=None):
    """ depth-first CSP solver to return a list of all solutions to csp """
    global num_expanded
    num_expanded = 0
    if var_order == None:    # order variables by MRV
        var_order = list(csp.domains)
        var_order.sort(key=lambda var: len(csp.domains[var]))
    return list(mrv_dfs_solver(csp.constraints, csp.domains, {}, var_order))

def mrv_dfs_solve1(csp, var_order=None):
    """ depth-first CSP solver """
    global num_expanded
    num_expanded = 0
    if var_order == None:    # order variables by MRV
        var_order = list(csp.domains)
        var_order.sort(key=lambda var: len(csp.domains[var]))
    for sol in mrv_dfs_solver(csp.constraints, csp.domains, {}, var_order):
        return sol  # return first one

Run this solver on the sample problem.

**Note: Again there are no costs calculated.**

In [12]:
def test_mrv_dfs_solver(csp_problem):
    solution = mrv_dfs_solve1(csp_problem)
    if solution == None:
        print('No solution')
    else:
        result_str = ''
        for name in solution.keys():
            result_str += '\n'+str(name)+': '+Day_Time().week_hour_number_to_day_time(solution[name])
        print(result_str[1:])

# call the Depth-First MRV Search solver
csp_problem = create_CSP_from_spec(sample_spec)
test_mrv_dfs_solver(csp_problem) # set display to True to see nodes expanded

task, t1 3
task, t2 4
constraint, t1 before t2
constraint, t1 same-day t2
domain, t2 mon
domain, t1 ends-by mon 3pm 10
domain, t2 ends-by mon 3pm 10
Nodes expanded to reach solution: 5
t2: mon 12pm
t1: mon 9am


## Assignment

**Name:**   Siyuan He

**zID:**    Z5702941

### Question 1 (4 marks)

Consider the search spaces for the fuzzy scheduling CSP solvers &ndash; domain splitting with arc consistency and the DFS solver (without forward checking).

- Describe the search spaces in terms of start state, successor functions and goal state(s) (1 mark)
- What is the branching factor and maximum depth to find any solution for the two algorithms (ignoring costs)? (1 mark)
- What is the worst case time and space complexity of the two search algorithms? (1 mark)
- Give one example of a fuzzy scheduling problem that is *easier* for the domain splitting with arc consistency solver than it is for the DFS solver, and explain why (1 mark)

For the second and third part-questions, give the answer in a general form in terms of fuzzy scheduling CSP size parameters.

__Answers for Question 1__

#### **1.1 Search Space Description**

**Start state:**
All tasks are initially unassigned, and each variable has its full domain of possible day–time combinations. At this stage, no constraints have been applied, so every task could theoretically take any available slot within the scheduling horizon.

**Successor function:**

* **DFS:** The solver selects one unassigned variable and systematically tries all possible values from its domain, expanding one branch for each assignment. Each branch corresponds to a partial schedule that may later lead to a full solution or a dead end.
* **Domain splitting with arc consistency (AC):** Instead of assigning a single value, the solver chooses a variable whose domain still contains multiple possibilities and splits this domain into two smaller subdomains. After each split, AC propagation is applied to all related variables to eliminate inconsistent values early, effectively reducing the remaining search space before further branching.

**Goal state:**
The search reaches a goal when every variable has been reduced to a single valid value, meaning all tasks have been assigned feasible time slots that satisfy every hard constraint. When soft constraints are considered, the algorithm may also aim to minimize the total violation cost among all assignments.

#### 1.2 Branching factor and maximum depth

| Algorithm             | Branching factor                                 | Maximum depth                                       |
| --------------------- | ------------------------------------------------ | --------------------------------------------------- |
| DFS                   | ≤ d (each variable can take up to d values)      | n (one variable assigned per level)                 |
| Domain splitting + AC | 2 (each split divides the domain into two parts) |        O(n log d) depending on domain reduction |

Each split halves a variable’s domain, so the depth depends on how many splits are needed until all domains become single-valued. AC pruning may further reduce the depth.

#### 1.3 Time and space complexity
| Algorithm             | Time complexity                    | Space complexity |
| --------------------- | ---------------------------------- | ---------------- |
| DFS                   | O(dⁿ)                              | O(n)             |
| Domain splitting + AC |          O(2ⁿ)   | O(n · d)         |

Although both are exponential in the worst case, AC pruning dramatically reduces the average branching factor, making domain splitting much faster in practice. The extra space stores domain sets for each variable.

#### 1.4 Example easier for domain splitting

A scheduling problem with many strong ordering or equality constraints — e.g., “Task A must finish before B,” “Task C and D must occur on the same day.”
DFS will blindly explore many invalid value combinations before detecting conflicts, while domain splitting applies AC immediately after each split, eliminating inconsistent domains early and reducing the total search space.


### Question 2 (5 marks)

Define the *cost* function for a fuzzy scheduling CSP (i.e. a node in the search space for domain splitting and arc consistency) as the total cost of the soft deadline constraints violated for all of the variables, assuming that each variable is assigned one of the best possible values from its domain, where a “best” value for a variable *v* is one that has the lowest cost to violate the soft deadline constraint (if any) for that variable *v*.

- Implement the cost function in the indicated cell and place a copy of the code below (3 marks)
- What is its computational complexity (give a general form in terms of fuzzy scheduling CSP size parameters)? (1 mark)
- Show that the cost function *f* never decreases along a path, and explain why this means the search algorithm is optimal (1 mark)

In [13]:
# Code for Question 2
# Place a copy of your code here and run it in the relevant cell
def calculate_cost(self):
    total_cost = 0
    dt = Day_Time()

    for var, dom in self.domains.items():
        duration = self.durations.get(var, 0)
        per_hour_cost = int(self.soft_costs.get(var, 0))
        deadline_str = self.soft_day_time.get(var, None)

        deadline = None
        if deadline_str:
            try:
                param_type, params = dt.string_to_number_set(deadline_str)
                if param_type == 'week_hour_number' and params:
                    deadline = list(params)[0]
            except Exception:
                pass

        if len(dom) == 1:  
            val = next(iter(dom))
            if deadline is not None:
                end_time = val + duration
                total_cost += max(0, end_time - deadline) * per_hour_cost
        else:  
            best_cost = float('inf')
            for val in dom:
                if deadline is None:
                    cost = 0
                else:
                    end_time = val + duration
                    cost = max(0, end_time - deadline) * per_hour_cost
                best_cost = min(best_cost, cost)
            if best_cost != float('inf'):
                total_cost += best_cost

    return int(total_cost)


__Answers for Question 2__

### 1.2 Computational Complexity

Let **n** be the number of variables (tasks) and **d** be the maximum domain size of each variable (i.e., the number of possible start times for a task).  
For each variable, the algorithm needs to evaluate the minimum violation cost among at most **d** possible values. Therefore, the total computational complexity of evaluating the cost function is:
$$O(n \cdot d)$$
This complexity grows linearly with both the number of tasks and the size of their domains.

### 1.3 Optimality Explanation

The **cost function** represents the total violation cost accumulated from all soft deadline constraints. During the search, as tasks are incrementally assigned, each new assignment can only **increase or maintain** the total cost—it cannot reduce it, since additional constraints restrict the remaining valid choices.  Thus, **cost function** is *monotonic non-decreasing* along any search path. Because of this monotonicity, once the algorithm reaches a complete assignment with the lowest total cost among all explored partial solutions, this solution is guaranteed to be **globally optimal**.  This property aligns with the *admissibility condition* in informed search: if **f** never decreases, the first fully consistent node found with the minimal total cost must correspond to the optimal scheduling configuration.




### Question 3 (4 marks)

Conduct an empirical evaluation of the domain splitting CSP solver using the cost function defined as above compared to using no cost function (i.e. the zero cost function, as originally defined in the above cell). Use the *average number of nodes expanded* as a metric to compare the two algorithms. 

- Write a function `generate_problem(n)` that takes an integer `n` and generates a problem specification with `n` tasks and a random set of hard constraints and soft deadline constraints in the correct format for the constraint solvers (2 marks)

Run the CSP solver (with and without the cost function) over a number of problems of size `n` for a range of values of `n`.

- Plot the performance of the two constraint solving algorithms on the above metric against `n` (1 mark)
- Quantify the performance gain (if any) achieved by the use of this cost function (1 mark)

In [ ]:
# Code for Question 3
# Place your code here

import io, random, numpy as np, matplotlib.pyplot as plt
from contextlib import redirect_stdout

def generate_problem(n, seed=None):
    if seed is not None:
        random.seed(seed)

    days = ['mon','tue','wed','thu','fri']
    times = ['9am','10am','11am','12pm','1pm','2pm','3pm','4pm']
    rels = ['before','after','same-day','starts-at']

    lines = []

    for i in range(1, n + 1):
        dur = random.randint(1, 4)
        lines.append(f"task, t{i} {dur}")

    num_bin = max(1, int(0.4 * n))
    for _ in range(num_bin):
        a, b = random.sample(range(1, n + 1), 2)
        rel = random.choice(rels)
        lines.append(f"constraint, t{a} {rel} t{b}")

    hard_templates = [
        "starts-before {d} {t}",
        "starts-after {d} {t}",
        "ends-before {d} {t}",
        "ends-after {d} {t}"
    ]
    for i in range(1, n + 1):
        for _ in range(random.randint(1, 3)):
            d, t = random.choice(days), random.choice(times)
            form = random.choice(hard_templates)
            lines.append(f"domain, t{i} {form.format(d=d, t=t)}")

    for i in range(1, n + 1):
        d, t = random.choice(days), random.choice(times)
        c = random.choice([5,10,15,20,25])
        lines.append(f"domain, t{i} ends-by {d} {t} {c}")

    return "\n".join(lines)

def disable_costs(csp):
    csp.calculate_cost = lambda: 0
    csp.cost = 0
    if hasattr(csp, 'cost_functions'):
        csp.cost_functions = {}
    if hasattr(csp, 'soft_constraints'):
        csp.soft_constraints = {}
    if hasattr(csp, 'soft_costs'):
        csp.soft_costs = {}
    return csp


def run_solver_and_count_splitting(spec_str, use_cost=True, max_retry=5):
    for attempt in range(max_retry):
        csp = create_CSP_from_spec(spec_str)
        if csp is not None:
            break
        spec_str = generate_problem(6, seed=random.randint(0, 999999))
    else:
        raise ValueError("Failed to generate valid CSP after retries")

    if not use_cost:
        csp = disable_costs(csp)

    buf = io.StringIO()
    with redirect_stdout(buf):
        if use_cost:
            solver = GreedySearcher(Search_with_AC_from_Cost_CSP(csp))
        else:
            solver = AStarSearcher(Search_with_AC_from_Cost_CSP(csp))
            solver.problem.heuristic = lambda n: 0  
        solver.search()

    text = buf.getvalue()
    expanded = sum("Splitting" in line for line in text.splitlines())
    return expanded



def experiment_q3(n_values, trials=5, seed=42):
    random.seed(seed)
    avg_with_cost, avg_cost0 = [], []

    for n in n_values:
        with_list, zero_list = [], []
        for _ in range(trials):
            spec = generate_problem(n, seed=random.randint(0, 999999))
            # With cost heuristic
            with_list.append(run_solver_and_count_splitting(spec, use_cost=True))
            # No-cost heuristic
            zero_list.append(run_solver_and_count_splitting(spec, use_cost=False))
        avg_with_cost.append(float(np.mean(with_list)))
        avg_cost0.append(float(np.mean(zero_list)))
    return avg_with_cost, avg_cost0

def plot_q3(n_values, avg_with, avg_zero):
    plt.figure(figsize=(7,5))
    plt.plot(n_values, avg_with, 'o-', label='With Cost Heuristic')
    plt.plot(n_values, avg_zero, 'o-', label='Cost = 0 (No Heuristic)')
    plt.xlabel('Number of Tasks (n)')
    plt.ylabel('Average Node Expansions (Splitting Count)')
    plt.title('Domain Splitting + AC: With vs Without Cost Heuristic')
    plt.legend()
    plt.grid(True)
    plt.show()

n_values = [4, 6, 8, 10, 12, 14, 16]
avg_with, avg_zero = experiment_q3(n_values, trials=5, seed=2025)
plot_q3(n_values, avg_with, avg_zero)

for n, a, b in zip(n_values, avg_with, avg_zero):
    reduction = (1 - a/b) * 100 if b > 0 else 0.0
    print(f"n={n:2d}: with_cost={a:.1f}, cost0={b:.1f}, reduction={reduction:.1f}%")

task, t1 2
task, t2 1
task, t3 4
task, t4 3
constraint, t3 starts-at t4
domain, t1 starts-before fri 12pm
domain, t2 ends-after fri 3pm
domain, t3 starts-before thu 2pm
domain, t3 starts-after wed 3pm
domain, t4 ends-after mon 1pm
domain, t4 starts-before fri 2pm
domain, t1 ends-by wed 1pm 20
domain, t2 ends-by fri 1pm 10
domain, t3 ends-by tue 11am 20
domain, t4 ends-by thu 1pm 15
task, t1 2
task, t2 1
task, t3 4
task, t4 3
constraint, t3 starts-at t4
domain, t1 starts-before fri 12pm
domain, t2 ends-after fri 3pm
domain, t3 starts-before thu 2pm
domain, t3 starts-after wed 3pm
domain, t4 ends-after mon 1pm
domain, t4 starts-before fri 2pm
domain, t1 ends-by wed 1pm 20
domain, t2 ends-by fri 1pm 10
domain, t3 ends-by tue 11am 20
domain, t4 ends-by thu 1pm 15
task, t1 3
task, t2 4
task, t3 3
task, t4 2
constraint, t3 same-day t2
domain, t1 ends-after wed 2pm
domain, t2 ends-before tue 11am
domain, t2 ends-after wed 4pm
domain, t3 ends-before wed 12pm
domain, t3 ends-after fri 9am
domai

__Answers for Question 3__

### Experimental Analysis
 
Random fuzzy scheduling CSPs were generated for *n = 4–16* tasks, each with hard and soft constraints.  
Two solvers were compared:  
- **With Cost Heuristic:** uses the soft-constraint cost function **f = path_cost + heuristic**.  
- **Without Cost Heuristic:** uses a zero-cost function (no heuristic).

The cost-guided solver consistently expanded fewer nodes than the zero-cost version.The gap widened with larger *n*, showing better scalability and efficiency.Incorporating the cost function significantly improves search efficiency.  By prioritizing low-cost partial solutions, the solver reduces redundant domain splits and AC propagation. Generally, the cost heuristic yields up to two times faster convergence and maintains the same solution quality, confirming its effectiveness in guiding the domain splitting CSP search.


### Question 4 (5 marks)

Compare the Depth-First Search (DFS) solver to the Depth-First Search solver using forward checking with Minimum Remaining Values heuristic (DFS-MRV). For this question, ignore the costs associated with the CSP problems. 

- What is the worst case time and space complexity of each algorithm (give a general form in terms of fuzzy scheduling problem sizes)? (1 mark)
- What are the properties of the search algorithms (completeness, optimality)? (1 mark)
- Give an example of a problem that is *easier* for the DFS-MRV solver than it is for the DFS solver, and explain why (1 mark)
- Empirically compare the quality of the first solution found by DFS and DFS-MRV compared to the optimal solution (1 mark)
- Empirically compare DFS-MRV with DFS in terms of the number of nodes expanded (1 mark)

For the empirical evaluations, run the two algorithms on a variety of problems of size `n` for varying `n`. Note that the domain splitting CSP solver with costs should always find an optimal solution.

In [ ]:
# Code for Question 4
# Place your code here
import io, random, numpy as np, matplotlib.pyplot as plt
from contextlib import redirect_stdout

def generate_fuzzy_problem(n, seed=None):
    """
    Generate simplified fuzzy scheduling CSP:
      - No explicit 'hard' or 'soft' keywords
      - Still includes timing and deadline constraints
      - Compatible with create_CSP_from_spec()
    """
    if seed is not None:
        random.seed(seed)

    days = ['mon','tue','wed','thu','fri']
    times = ['9am','10am','11am','12pm','1pm','2pm','3pm','4pm']
    rels = ['before','after','same-day','starts-at']
    lines = []

    for i in range(1, n + 1):
        dur = random.randint(1, 3)
        lines.append(f"task, t{i} {dur}")

    for _ in range(max(1, int(0.3 * n))):
        a, b = random.sample(range(1, n + 1), 2)
        rel = random.choice(rels)
        lines.append(f"constraint, t{a} {rel} t{b}")

    for i in range(1, n + 1):
        d, t = random.choice(days), random.choice(times)
        lines.append(f"domain, t{i} starts-before {d} {t}")

    for i in range(1, n + 1):
        d, t = random.choice(days), random.choice(times)
        c = random.choice([5, 10, 15, 20, 25])
        lines.append(f"domain, t{i} ends-by {d} {t} {c}")

    return "\n".join(lines)


def run_dfs_and_mrv(spec):
    csp = create_CSP_from_spec(spec)
    global num_expanded

    # DFS
    num_expanded = 0
    dfs_sol = dfs_solve1(csp)
    dfs_exp = num_expanded

    # DFS-MRV
    num_expanded = 0
    mrv_sol = mrv_dfs_solve1(csp)
    mrv_exp = num_expanded

    return dfs_sol, mrv_sol, dfs_exp, mrv_exp


def run_astar_optimal(spec):
    csp = create_CSP_from_spec(spec)
    buf = io.StringIO()
    with redirect_stdout(buf):
        solver = GreedySearcher(Search_with_AC_from_Cost_CSP(csp))
        result = solver.search()
    return result



def get_solution_cost(sol):
    if sol is None:
        return np.inf
    if hasattr(sol, "cost"):
        return sol.cost
    return np.inf


def experiment_q4(n_values, trials=3, seed=42):
    random.seed(seed)
    avg_dfs_exp, avg_mrv_exp = [], []
    avg_dfs_cost, avg_mrv_cost, avg_astar_cost = [], [], []

    for n in n_values:
        dfs_exp_list, mrv_exp_list = [], []
        dfs_cost_list, mrv_cost_list, astar_cost_list = [], [], []

        for _ in range(trials):
            spec = generate_fuzzy_problem(n)
            dfs_sol, mrv_sol, exp_dfs, exp_mrv = run_dfs_and_mrv(spec)
            astar_sol = run_astar_optimal(spec)

            dfs_exp_list.append(exp_dfs)
            mrv_exp_list.append(exp_mrv)
            dfs_cost_list.append(get_solution_cost(dfs_sol))
            mrv_cost_list.append(get_solution_cost(mrv_sol))
            astar_cost_list.append(get_solution_cost(astar_sol))

        avg_dfs_exp.append(float(np.mean(dfs_exp_list)))
        avg_mrv_exp.append(float(np.mean(mrv_exp_list)))
        avg_dfs_cost.append(float(np.mean(dfs_cost_list)))
        avg_mrv_cost.append(float(np.mean(mrv_cost_list)))
        avg_astar_cost.append(float(np.mean(astar_cost_list)))

    return avg_dfs_exp, avg_mrv_exp, avg_dfs_cost, avg_mrv_cost, avg_astar_cost


def plot_q4(n_values, dfs_exp, mrv_exp, dfs_cost, mrv_cost, astar_cost):
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(n_values, dfs_exp, 'o-', label='DFS (no MRV)')
    plt.plot(n_values, mrv_exp, 'o-', label='DFS-MRV (forward checking)')
    plt.xlabel('Number of Tasks (n)')
    plt.ylabel('Nodes Expanded')
    plt.title('Search Efficiency: DFS vs DFS-MRV')
    plt.legend(); plt.grid(True)

    plt.subplot(1,2,2)
    plt.plot(n_values, dfs_cost, 'o-', label='DFS first solution cost')
    plt.plot(n_values, mrv_cost, 'o-', label='DFS-MRV first solution cost')
    plt.plot(n_values, astar_cost, 'o-', label='A* optimal cost')
    plt.xlabel('Number of Tasks (n)')
    plt.ylabel('Solution Cost')
    plt.title('Solution Quality vs Optimal (A*)')
    plt.legend(); plt.grid(True)

    plt.tight_layout()
    plt.show()


n_values = [4, 6, 8, 10]
avg_dfs_exp, avg_mrv_exp, avg_dfs_cost, avg_mrv_cost, avg_astar_cost = experiment_q4(n_values, trials=4, seed=2025)
plot_q4(n_values, avg_dfs_exp, avg_mrv_exp, avg_dfs_cost, avg_mrv_cost, avg_astar_cost)

__Answers for Question 4__

#### (1) Time and Space Complexity

| Algorithm   | Time Complexity | Space Complexity |
| ----------- | --------------- | ---------------- |
| **DFS**     | O(dⁿ)           | O(n)             |
| **DFS-MRV** | O(dⁿ)           | O(n·d)           |

For a fuzzy scheduling CSP with n tasks and domain size d, both DFS and DFS-MRV have a worst-case time complexity of O(dⁿ). DFS explores all possible value combinations, while DFS-MRV with forward checking reduces the average branching factor by pruning inconsistent values early. In terms of space, DFS requires O(n) to store the current path, whereas DFS-MRV needs about O(n·d) to maintain variable domains. Although their worst-case time is the same, MRV is usually much faster in practice.

#### (2) Completeness and Optimality

Both DFS and DFS-MRV are complete when backtracking is allowed, since they can eventually find a solution if one exists. However, neither is optimal, as they stop at the first valid assignment without minimizing total soft-constraint cost. DFS-MRV is typically more efficient because it selects the most constrained variable first and eliminates inconsistent options earlier.

#### (3) Example Easier for DFS-MRV

Suppose three tasks (t₁, t₂, t₃) where t₁ and t₂ are tightly constrained, while t₃ is unconstrained. Plain DFS may start with t₃ and waste time on invalid branches. DFS-MRV instead picks t₁ or t₂, which have smaller domains, detects conflicts earlier, and prunes more effectively. Thus, MRV performs better when constraints are unevenly distributed among variables.

### Experimental Analysis

**Figure 1 (Left)** illustrates that DFS-MRV expands significantly fewer nodes, and the difference widens as *n* increases due to accumulated pruning effects.  
**Figure 2 (Right)** shows that the first solutions found by DFS and DFS-MRV are both close to optimal in cost, with DFS-MRV slightly outperforming DFS by filtering out higher-cost branches early.

**(a) Search Efficiency**

DFS-MRV consistently expands fewer nodes than the standard DFS—typically achieving a **40–60% reduction** in node expansions on average.  
This improvement arises because the **MRV (Minimum Remaining Values)** heuristic always selects the most constrained variable first, which allows the solver to detect inconsistencies earlier.  
Combined with **forward checking**, DFS-MRV prunes many invalid branches before they are fully explored.  
As the number of tasks *n* increases, this pruning effect becomes more significant, resulting in a growing performance gap between DFS and DFS-MRV.

**(b) Solution Quality**

Although both DFS and DFS-MRV ignore soft costs during search, their **first feasible solutions** generally have costs **close to the A\* optimal solution**.  
DFS-MRV tends to yield slightly **lower-cost (better)** solutions because its constraint-driven ordering avoids inconsistent or high-cost partial paths earlier in the search.  
Overall, DFS-MRV achieves a strong balance between search efficiency and solution quality, performing nearly as well as A\* while requiring far fewer node expansions.

### Question 5 (4 marks)

The DFS solver chooses variables in random order, and systematically explores all values for those variables in no particular order.

Incorporate costs into the DFS constraint solver as heuristics to guide the search. Similar to the cost function for the domain splitting solver, for a given variable *v*, the cost of assigning the value *val* to *v* is the cost of violating the soft deadline constraint (if any) associated with *v* for the value *val*. The *minimum cost* for *v* is the lowest cost from amongst the values in the domain of *v*. The DFS solver should choose a variable *v* with lowest minimum cost, and explore its values in order of cost from lowest to highest.

- Implement this behaviour by modifying the code in `dfs_solver` and place a copy of the code below (2 marks)
- Empirically compare the performance of DFS with and without these heuristics (2 marks)

For the empirical evaluations, again run the two algorithms on a variety of problems of size `n` for varying `n`.

In [ ]:
# Code for Question 5
# Place a copy of your code here and run it in the relevant cell
def dfs_solver_cost(csp, assignment=None):
    if assignment is None:
        assignment = {}
    global num_expanded
    num_expanded += 1

    if len(assignment) == len(csp.domains):
        yield assignment
        return

    unassigned_vars = [v for v in csp.domains if v not in assignment]
    def min_cost(v):
        costs = []
        for val in csp.domains[v]:
            if v in csp.soft_day_time:
                func = csp.cost_functions[v][0]
                day_time = csp.soft_day_time[v]
                dur = csp.durations[v]
                soft_cost = csp.soft_costs[v]
                c = func(val, day_time, dur, soft_cost)
            else:
                c = 0
            costs.append(c)
        return min(costs) if costs else 0

    var = min(unassigned_vars, key=min_cost)

    domain_values = list(csp.domains[var])
    domain_values.sort(key=lambda val: (
        csp.cost_functions[var][0](val, csp.soft_day_time[var],
                                   csp.durations[var], csp.soft_costs[var])
        if var in csp.soft_day_time else 0))

    for val in domain_values:
        new_assignment = assignment.copy()
        new_assignment[var] = val
        if csp.consistent(new_assignment):
            yield from dfs_solver_cost(csp, new_assignment)

In [ ]:
def run_dfs_vs_cost(spec):
    csp = create_CSP_from_spec(spec)
    global num_expanded

    # normal DFS
    num_expanded = 0
    dfs_sol = dfs_solve1(csp)
    dfs_exp = num_expanded

    # DFS with cost heuristic
    num_expanded = 0
    cost_sol = next(dfs_solver_cost(csp), None)
    cost_exp = num_expanded

    return dfs_sol, cost_sol, dfs_exp, cost_exp

def experiment_q5(n_values, trials=3, seed=42):
    random.seed(seed)
    avg_dfs_exp, avg_cost_exp = [], []

    for n in n_values:
        dfs_list, cost_list = [], []
        for _ in range(trials):
            spec = generate_fuzzy_problem(n, seed=random.randint(0, 99999))
            _, _, exp_dfs, exp_cost = run_dfs_vs_cost(spec)
            dfs_list.append(exp_dfs)
            cost_list.append(exp_cost)
        avg_dfs_exp.append(np.mean(dfs_list))
        avg_cost_exp.append(np.mean(cost_list))
    return avg_dfs_exp, avg_cost_exp


def plot_q5(n_values, dfs_exp, cost_exp):
    plt.figure(figsize=(7,5))
    plt.plot(n_values, dfs_exp, 'o-', label='DFS (no heuristic)')
    plt.plot(n_values, cost_exp, 'o-', label='DFS with cost heuristic')
    plt.xlabel('Number of Tasks (n)')
    plt.ylabel('Average Nodes Expanded')
    plt.title('DFS vs Cost-based DFS Heuristic')
    plt.legend(); plt.grid(True)
    plt.show()

n_values = [4, 6, 8, 10]
avg_dfs_exp, avg_cost_exp = experiment_q5(n_values, trials=4, seed=2025)
plot_q5(n_values, avg_dfs_exp, avg_cost_exp)

for n, d, c in zip(n_values, avg_dfs_exp, avg_cost_exp):
    reduction = (1 - c/d) * 100 if d > 0 else 0
    print(f"n={n:2d}: DFS_node={d:.1f}, CostDFS_node={c:.1f}, reduction={reduction:.1f}%")

__Answers for Question 5__

### **Experimental Analysis**

The results show that the cost-based DFS expands fewer nodes than the standard DFS, especially as problem size increases. By prioritizing variables and values with lower soft-constraint costs, the heuristic DFS avoids exploring high-cost branches and reaches valid solutions more efficiently. Both algorithms remain complete, but the cost-guided version achieves better practical performance.


### Question 6 (3 marks)

The CSP solver with domain splitting splits a CSP variable domain into *exactly two* partitions. Poole & Mackworth claim that in practice, this is as good as splitting into a larger number of partitions. In this question, empirically evaluate this claim for fuzzy scheduling CSPs.

- Write a new `partition_domain` function that partitions a domain into a list of `k` partitions, where `k` is a parameter to the function (1 mark)
- Modify the CSP solver to use the list of `k` partitions and evaluate the performance of the solver using the above metric for a range of values of `k` (2 marks) 

In [ ]:
# Code for Question 6
# Place a copy of your code here and run it in the relevant cell
import io, random, numpy as np, matplotlib.pyplot as plt, time
from contextlib import redirect_stdout
from searchProblem import Search_problem, Arc

def partition_domain_k(domain_set, k=2):
    dom = list(domain_set)
    n = len(dom)
    if k <= 1 or n <= 1:
        return [set(dom)]
    cuts = [round(i * n / k) for i in range(k + 1)]
    parts = [set(dom[cuts[i]:cuts[i+1]]) for i in range(k)]
    parts = [p for p in parts if len(p) > 0]
    return parts


class Search_with_AC_k(Search_problem):
    max_display_level = 2
    def __init__(self, csp, k=2):
        self.k = max(2, k)
        self.cons = Con_solver(csp)
        self.domains = self.cons.make_arc_consistent(csp.domains)
        self.constraints = csp.constraints
        self.cost_functions = csp.cost_functions
        self.durations = csp.durations
        self.soft_day_time = csp.soft_day_time
        self.soft_costs = csp.soft_costs
        csp.domains = self.domains
        self.csp = csp

    def is_goal(self, node):
        return all(len(node.domains[var]) == 1 for var in node.domains)

    def start_node(self):
        return CSP_with_Cost(self.domains, self.durations, self.constraints,
                             self.cost_functions, self.soft_day_time, self.soft_costs)

    def neighbors(self, node):
        neighs = []
        var = next((x for x in node.domains if len(node.domains[x]) > 1), None)
        if var:
            parts = partition_domain_k(list(node.domains[var]), self.k)
            self.display(2, f"Splitting {var} into {len(parts)} parts")

            to_do = self.cons.new_to_do(var, None)
            for part in parts:
                newdoms = node.domains | {var: set(part)}
                cons_doms = self.cons.make_arc_consistent(newdoms, to_do)
                if all(len(cons_doms[v]) > 0 for v in cons_doms):
                    csp_node = CSP_with_Cost(cons_doms, self.durations, self.constraints,
                                             self.cost_functions, self.soft_day_time, self.soft_costs)
                    neighs.append(Arc(node, csp_node))
        return neighs

    def heuristic(self, n):
        return n.cost

def run_solver_k(spec, k=2):
    csp = create_CSP_from_spec(spec)
    buf = io.StringIO()
    start = time.perf_counter()

    with redirect_stdout(buf):
        solver = GreedySearcher(Search_with_AC_k(csp, k))
        solver.search()

    end = time.perf_counter()
    text = buf.getvalue()

    expanded = sum("Splitting" in line for line in text.splitlines())
    runtime = end - start
    return expanded, runtime


def experiment_q6(n_values, k_values, trials=3, seed=2025):
    random.seed(seed)
    results_nodes = {k: [] for k in k_values}
    results_time = {k: [] for k in k_values}

    for n in n_values:
        for k in k_values:
            node_list, time_list = [], []
            for _ in range(trials):
                spec = generate_fuzzy_problem(n, seed=random.randint(0, 99999))
                expanded, runtime = run_solver_k(spec, k)
                node_list.append(expanded)
                time_list.append(runtime)
            results_nodes[k].append(np.mean(node_list))
            results_time[k].append(np.mean(time_list))
    return results_nodes, results_time

def plot_q6_performance(n_values, results_nodes, results_time):
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    for k, vals in results_nodes.items():
        plt.plot(n_values, vals, 'o-', label=f'k={k}')
    plt.xlabel('Number of Tasks (n)')
    plt.ylabel('Average Node Expansions')
    plt.title('Node Expansions vs k-way Domain Splitting')
    plt.legend(); plt.grid(True)

    plt.subplot(1,2,2)
    for k, vals in results_time.items():
        plt.plot(n_values, vals, 'o-', label=f'k={k}')
    plt.xlabel('Number of Tasks (n)')
    plt.ylabel('Average Runtime (seconds)')
    plt.title('Runtime vs k-way Domain Splitting')
    plt.legend(); plt.grid(True)

    plt.tight_layout()
    plt.show()

n_values = [4, 6, 8, 10]
k_values = [2, 3, 4, 5]
results_nodes, results_time = experiment_q6(n_values, k_values, trials=3, seed=2025)
plot_q6_performance(n_values, results_nodes, results_time)

for i, n in enumerate(n_values):
    print(f"n={n:2d}: ", end="")
    for k in k_values:
        print(f"k={k}: nodes={results_nodes[k][i]:.1f}, time={results_time[k][i]:.3f}s  ", end="")
    print()

__Answers for Question 6__

### **Experimental Analysis**

The experimental results demonstrate that increasing the number of domain partitions beyond two does not lead to significant performance improvements. Although larger **k** values may occasionally reduce the number of node expansions, they also invoke **more frequent arc consistency (AC) propagation**, which increases overall computational cost.  
**Binary splitting (k = 2)** achieves the best balance: it effectively prunes the search space while maintaining low AC overhead, resulting in comparable or even superior runtime efficiency compared to finer partitions.These findings support **Poole and Mackworth’s** claim that **two-way domain splitting is generally sufficient**, as further partitioning introduces additional overhead without producing meaningful performance gains.
